# Molecular Dynamics


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from ase.io import read, write
from ase import units
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import Stationary, ZeroRotation, MaxwellBoltzmannDistribution
from xtb.ase.calculator import XTB
from mace.calculators import MACECalculator

# Setup for directories
output_dir = "moldyn"
os.makedirs(output_dir, exist_ok=True)

# Simulation function that logs temperature and energy
def run_md(name, init_conf, temp, calc, steps, interval):
    conf = init_conf.copy()
    conf.set_calculator(calc)

    # Initialize velocities and remove translation/rotation
    MaxwellBoltzmannDistribution(conf, temperature_K=300)
    Stationary(conf)
    ZeroRotation(conf)

    dyn = Langevin(conf, 1.0 * units.fs, temperature_K=temp, friction=0.1)
    traj_file = os.path.join(output_dir, f"{name}.xyz")
    if os.path.exists(traj_file):
        os.remove(traj_file)

    time_fs, temperatures, energies = [], [], []

    def log():
        dyn.atoms.write(traj_file, append=True)
        time_fs.append(dyn.get_time() / units.fs)
        temperatures.append(dyn.atoms.get_temperature())
        energies.append(dyn.atoms.get_potential_energy() / len(dyn.atoms))

    dyn.attach(log, interval=interval)
    dyn.run(steps)

    return np.array(time_fs), np.array(temperatures), np.array(energies)

# Load starting configuration: one molecule from solvent_molecs.xyz
init_conf = read('data/solvent_molecs.xyz', ':')
init_conf = [a for a in init_conf if a.info.get('Nmols') == 1][0]

# Run MACE dynamics
mace_calc = MACECalculator(model_paths=['MACE_models/mace_learncurve_train4000_swa_compiled.model'],
                           device='cuda', default_dtype='float32')
mace_t, mace_temp, mace_E = run_md("mace_md_molecule", init_conf, temp=1200, calc=mace_calc, steps=5000, interval=10)

# Run XTB dynamics
xtb_calc = XTB(method="GFN2-xTB")
xtb_t, xtb_temp, xtb_E = run_md("xtb_md_molecule", init_conf, temp=1200, calc=xtb_calc, steps=5000, interval=10)

# Plot energy comparison
plt.figure(figsize=(6, 4))
plt.plot(mace_t, mace_E, label='MACE', color='blue')
plt.plot(xtb_t, xtb_E, label='XTB', color='red')
plt.xlabel("Time (fs)")
plt.ylabel("Energy (eV/atom)")
plt.title("Potential Energy per Atom")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "energy_comparison.png"), dpi=300)
plt.close()

# Plot temperature with average lines
plt.figure(figsize=(6, 4))
plt.plot(mace_t, mace_temp, label='MACE T(t)', color='blue')
plt.plot(xtb_t, xtb_temp, label='XTB T(t)', color='red')
plt.axhline(np.mean(mace_temp), linestyle='--', color='blue', label=f'MACE ⟨T⟩ = {np.mean(mace_temp):.1f} K')
plt.axhline(np.mean(xtb_temp), linestyle='--', color='red', label=f'XTB ⟨T⟩ = {np.mean(xtb_temp):.1f} K')
plt.xlabel("Time (fs)")
plt.ylabel("Temperature (K)")
plt.title("Temperature Over Time")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "temperature_comparison.png"), dpi=300)
plt.close()

import aseMolec.anaAtoms as aa

# Function to plot RDFs for multiple tags
def plot_rdfs(file_xtb, file_mace, tags, skip=50):
    traj_xtb = read(file_xtb, f'{skip}:')
    traj_mace = read(file_mace, f'{skip}:')

    for traj in [traj_xtb, traj_mace]:
        for at in traj:
            at.pbc = True
            at.cell = [100, 100, 100]

    for tag in tags:
        rdf_xtb = aa.compute_rdfs_traj_avg(traj_xtb, rmax=5, nbins=50)
        rdf_mace = aa.compute_rdfs_traj_avg(traj_mace, rmax=5, nbins=50)

        plt.figure(figsize=(5, 3.5))
        plt.plot(rdf_xtb[1], rdf_xtb[0][tag], label="XTB", alpha=0.7)
        plt.plot(rdf_mace[1], rdf_mace[0][tag], label="MACE", alpha=0.7)
        plt.xlabel(r"R ($\rm \AA$)")
        plt.ylabel(f"RDF {tag}")
        plt.legend()
        plt.title(f"RDF {tag}")
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"rdf_{tag}.png"), dpi=300)
        plt.close()

# Plot RDFs
rdf_tags = ['HH_intra', 'HC_intra', 'HO_intra', 'CC_intra', 'CO_intra', 'OO_intra']
plot_rdfs('moldyn/xtb_md_molecule.xyz', 'moldyn/mace_md_molecule.xyz', rdf_tags)



/usr/local/lib/python3.10/dist-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.d

Default dtype float32 does not match model dtype float64, converting models to float32.


/tmp/ipykernel_2157033/954310160.py:18: DeprecationWarning: Please use atoms.calc = calc
  conf.set_calculator(calc)


## Molec. Liquid

Try to simulate a liquid with the model (XTB can't do these boundary conditions) and evaluate the results.


In [2]:
from ase.io import read
from mace.calculators import MACECalculator

# Read the periodic input configuration
init_conf = read("data/input2.xyz")
init_conf.center()  # recenter atoms (optional)

# Assign MACE calculator (float64 recommended for accuracy)
mace_calc = MACECalculator(
    model_paths=["MACE_models/mace_learncurve_train4000_swa_compiled.model"],
    device="cuda", default_dtype="float64"
)

# Run dynamics for 5 ps at 500 K (T01 suggested this setup)
mace_t, mace_temp, mace_E = run_md(
    name="mace_md_input2", 
    init_conf=init_conf, 
    temp=500, 
    calc=mace_calc, 
    steps=5000,  # 5 ps
    interval=10
)


/tmp/ipykernel_2157033/954310160.py:18: DeprecationWarning: Please use atoms.calc = calc
  conf.set_calculator(calc)


In [3]:
from ase.io import read
from aseMolec import anaAtoms as aa

traj = read("moldyn/mace_md_input2.xyz", "50:")  # skip initial frames

# Set fake box if necessary (if ASE complains)
for at in traj:
    at.pbc = True
    at.cell = [100, 100, 100]

rdf = aa.compute_rdfs_traj_avg(traj, rmax=5, nbins=50)

# Plot RDF for some intermolecular tags
for tag in ['HO_inter', 'OO_inter', 'CC_inter']:
    plt.plot(rdf[1], rdf[0][tag], label=tag)
    plt.xlabel(r"R ($\rm \AA$)")
    plt.ylabel(f"RDF {tag}")
    plt.legend()
    plt.savefig(f"moldyn/input2_rdf_{tag}.png", dpi=300)
    plt.close()


In [5]:
!python3 MSD.py moldyn/mace_md_input2.xyz --out moldyn/msd_mace_input2.dat --dt 1 --png moldyn/msd_mace_input2.png --skip 1

✔  wrote moldyn/msd_mace_input2.dat  with 501 lines
✔  wrote moldyn/msd_mace_input2.png
